In [ ]:
import tensorflow as tf

import os, pathlib, warnings, urllib, shutil

warnings.filterwarnings("ignore")

In [ ]:
giturl = "https://github.com/dicodingacademy/assets/releases/download/release/"

dataset = "rockpaperscissors.zip"

urllib.request.urlretrieve(os.path.join(giturl, dataset), dataset)

shutil.unpack_archive('rockpaperscissors.zip', '/content', 'zip')

In [ ]:
root = "/content/rockpaperscissors/rps-cv-images"

os.listdir(root)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

rescale = 1.0 / 255.0

trainset = ImageDataGenerator(
  rescale = rescale,
  fill_mode = 'nearest'
)

validset = ImageDataGenerator(rescale=rescale)

In [ ]:
total_batch = 128

train = trainset.flow_from_directory(
  root,
  target_size = (150, 150),
  class_mode = "categorical",
  subset = "training",
  batch_size = total_batch
)

In [ ]:
valid = validset.flow_from_directory(
  root,
  target_size = (150, 150),
  class_mode = "categorical",
  subset = "validation",
  batch_size = total_batch
)